In [2]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from sklearn.preprocessing import LabelEncoder 
  
le = LabelEncoder() 

In [124]:
data_original  = pd.read_csv('donors.csv', low_memory=False)

data = data_original.copy()

data.dtypes

Unnamed: 0     int64
ODATEDW       object
OSOURCE       object
TCODE          int64
STATE         object
               ...  
RFA_2A        object
MDMAUD_R      object
MDMAUD_F      object
MDMAUD_A      object
GEOCODE2      object
Length: 476, dtype: object

In [4]:
data.head(5)

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,...,7.741935,95515,0,L,4,E,X,X,X,C
1,1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2,2010-01-01,AMH,1,NC,27017,,,NaN,0,...,7.481481,15078,1,L,4,E,X,X,X,C
3,3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,...,6.812500,172556,1,L,4,E,X,X,X,C
4,4,2006-01-01,,0,FL,33176,,,1940-01-01,0,...,6.864865,7112,1,L,2,F,X,X,X,A


In [5]:
data = data.rename(columns={'Unnamed: 0': 'ID'})
data.set_index('ID', inplace=True)
data.head(5)

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
ID,,,,,,,,,,,,,,,,,,,,,
0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,...,7.741935,95515,0,L,4,E,X,X,X,C
1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2010-01-01,AMH,1,NC,27017,,,NaN,0,,...,7.481481,15078,1,L,4,E,X,X,X,C
3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,,...,6.812500,172556,1,L,4,E,X,X,X,C
4,2006-01-01,,0,FL,33176,,,1940-01-01,0,X,...,6.864865,7112,1,L,2,F,X,X,X,A


In [69]:
#COLUMN NOEXCH CONTAINS 0's and 1's , possible values are '_' and 'X' . Will we drop this column ?
data.groupby('NOEXCH').count()

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,RECINHSE,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
NOEXCH,,,,,,,,,,,,,,,,,,,,,
,7,7,7,7,7,7,7,7,2,7,...,7,7,7,7,7,7,7,7,7,7
0,95085,95085,95085,95085,95085,95085,95085,95085,71303,95085,...,95085,95085,95085,95085,95085,95085,95085,95085,95085,94953
1,285,285,285,285,285,285,285,285,200,285,...,285,285,285,285,285,285,285,285,285,285
X,35,35,35,35,35,35,35,35,24,35,...,35,35,35,35,35,35,35,35,35,35


In [72]:
# ONE HOT ENCODE COLUMNS
one_hot_encode=["MAILCODE", "PVASTATE", "RECINHSE" , "RECP3" , "RECPGVG" , "RECSWEEP" , "CHILD03" , "CHILD07" ,
                "CHILD12" , "CHILD18" , "HOMEOWNR" ]

for col_name in data.columns:
    if col_name in one_hot_encode :
        col_dummies = pd.get_dummies(data[col_name], prefix=col_name)
        data = pd.merge(data, col_dummies, left_index=True, right_index=True)
        


In [81]:
# DROP THE COLUMNS WE USED ABOVE TO HOT ENCODE
data = data.drop(one_hot_encode, axis = 1)

In [84]:
#DROP COLUMN TCODE
data = data.drop(['TCODE'], axis = 1)

In [79]:
#APPLY LABEL ENCODING -> Give a numeric value for each string
label_encoding = ['STATE' , 'OSOURCE']
for x in label_encoding:
    data[x]= le.fit_transform(data[x]) 

In [85]:
# ODATEDW -> Calculate number of days between date and today's date

,Unnamed: 0,ODATEDW,OSOURCE,STATE,ZIP,DOB,NOEXCH,MDMAUD,DOMAIN,HOMEOWNR,...,CHILD07_F,CHILD07_M,CHILD12_,CHILD12_B,CHILD12_F,CHILD12_M,CHILD18_,CHILD18_B,CHILD18_F,CHILD18_M
0,0,2009-01-01,342,19,61081,1957-12-01,0,XXXX,T2,,...,0,0,1,0,0,0,1,0,0,0
1,1,2014-01-01,121,8,91326,1972-02-01,0,XXXX,S1,H,...,0,0,1,0,0,0,0,0,0,1
2,2,2010-01-01,49,32,27017,NaN,0,XXXX,R2,U,...,0,0,1,0,0,0,1,0,0,0
3,3,2007-01-01,127,8,95953,1948-01-01,0,XXXX,R2,U,...,0,0,1,0,0,0,1,0,0,0
4,4,2006-01-01,0,13,33176,1940-01-01,0,XXXX,S2,H,...,0,0,1,0,0,0,1,0,0,0


In [92]:
data.ZIP.unique()

array(['61081', '91326', '27017', ..., '55932', '92581', '32776-'],
      dtype=object)

In [118]:
#ZIP Convert do INTEGER so we have the column values as numbers. There's 5k rows with an '-' after the number value like '37127-' . We will drop rows where ZIP
# is not numeric (LOSING 5k ROWS (?) ) or can we treat the rows and remove the '-'
data.ZIP = data.ZIP.astype(str)
data.ZIP = data.ZIP.apply(lambda x: x.replace('-', '') if x.find('-') != -1 else x )
data = data[data.ZIP.apply(lambda x: x.isnumeric())]
data.ZIP = data.ZIP.astype(int)

In [93]:
# GENDER - Male , Female , Others
data.GENDER = data.GENDER.apply(lambda x: 'M' if x == 'M' else 'F'if x == 'F' else 'O')

In [97]:
data.NUMCHLD.unique()

array([nan,  1.,  2.,  3.,  4.,  5.,  6.,  7.])

In [98]:
# NUMCHILD -> There's no 0 values ?? Unknown values will be setted to 0 
data.NUMCHLD.replace(np.nan, 0 , inplace=True)

In [99]:
data.NUMCHLD.unique()

array([0., 1., 2., 3., 4., 5., 6., 7.])

In [120]:
# INCOME -> There's no 0 values ?? Some people don't like to speak about monetary situation, but for
# testing purposes unknown values will be setted to 0 
data.INCOME.replace(np.nan, 0 , inplace=True)

In [129]:
# Interests section
interests = ['COLLECT1' , 'VETERANS' , 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO'
            , 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS' , 'WALKER' , 'KIDSTUFF' , 'CARDS' , 'PLATES' , 'LIFESRC' , 'PEPSTRFL']

for col_name in data.columns:
    if col_name in interests :
        col_dummies = pd.get_dummies(data[col_name], prefix=col_name)
        data = pd.merge(data, col_dummies, left_index=True, right_index=True)


array([' ', '3', '1', '2'], dtype=object)

In [140]:
# responded to other types of mail order offers. Will change this logic to responded or not responde . NaN or 0 -> N , else Y

other_offers = [ 'MBCRAFT' , 'MBGARDEN' , 'MBBOOKS' , 'MBCOLECT' , 'MAGFAML' , 'MAGFEM' , 'MAGMALE' , 'PUBGARDN' , 'PUBCULIN' , 'PUBHLTH' , 'PUBDOITY' ,
               'PUBNEWFN' , 'PUBPHOTO' , 'PUBOPP' ]

for col_name in other_offers:
    data[col_name] = data[col_name].apply(lambda x: 'N' if x == np.nan or x == 0 else 'Y')

In [142]:
for col_name in other_offers:
        col_dummies = pd.get_dummies(data[col_name], prefix=col_name)
        data = pd.merge(data, col_dummies, left_index=True, right_index=True)
        
# DROP THE COLUMNS WE USED ABOVE TO HOT ENCODE
data = data.drop(other_offers, axis = 1)

In [143]:
#Interests section
interests = [ 'COLLECT1' , 'VETERANS' , 'BIBLE' , 'CATLG' , 'HOMEE' , 'PETS' , 'CDPLAY' , 'STEREO' , 'PCOWNERS' , 'PHOTO' ,
             'CRAFTS' , 'FISHER' , 'GARDENIN' , 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES' ]

for col_name in interests:
    data[col_name] = data[col_name].apply(lambda x: 'Y' if x =='Y' else 'N')
    
for col_name in interests:
        col_dummies = pd.get_dummies(data[col_name], prefix=col_name)
        data = pd.merge(data, col_dummies, left_index=True, right_index=True)
        
data = data.drop(interests, axis = 1)
    

In [148]:
data['LIFESRC']= le.fit_transform(data['LIFESRC']) 

In [145]:
#PEPStar 
col_dummies = pd.get_dummies(data.PEPSTRFL, prefix=col_name)
data = pd.merge(data, col_dummies, left_index=True, right_index=True)
        
data = data.drop(['PEPSTRFL'], axis = 1)

In [161]:
# MAJOR 
col_dummies = pd.get_dummies(data.MAJOR, prefix=col_name)
data = pd.merge(data, col_dummies, left_index=True, right_index=True)
        
data = data.drop(['MAJOR'], axis = 1)

In [162]:
# FIT TRANSFORM 
lastFitTransformation = [ 'DATASRCE' , 'SOLP3' , 'SOLIH' , 'WEALTH2' , 'GEOCODE' , 'WEALTH1' , 'HIT'  ]

for x in lastFitTransformation:
    data[x]= le.fit_transform(data[x]) 

In [163]:
# COLUMNS TO DROP 
dropCols = [ 'MALEMILI' , 'MALEVET' , 'VIETVETS' , 'WWIIVETS' , 'LOCALGOV' , 'STATEGOV' , 'FEDGOV']
data = data.drop(dropCols, axis = 1)

In [167]:
#MDMAUD Splitting
data['MDMAUD_1'] = data.MDMAUD.astype(str).str[0]
data['MDMAUD_2'] = data.MDMAUD.astype(str).str[1]
data['MDMAUD_3'] = data.MDMAUD.astype(str).str[2]
data['MDMAUD_4'] = data.MDMAUD.astype(str).str[3]

newMdmaud = ['MDMAUD_1','MDMAUD_2','MDMAUD_3','MDMAUD_4']
for x in newMdmaud:
    data[x]= le.fit_transform(data[x]) 

In [169]:
#DOMAIN
data['DOMAIN_1'] = data.DOMAIN.astype(str).str[0]
data['DOMAIN_2'] = data.DOMAIN.astype(str).str[1]

data['DOMAIN_2'] = ['4' if a != 'U' and b == '3' else b for a, b in zip(data['DOMAIN_1'] , data['DOMAIN_2'] ) ]
